## <span style="color:#ff5f27">🧬 Train Retrieval Model </span>


## <span style="color:#ff5f27">📝 Imports </span>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import StringLookup, Normalization

import tensorflow_recommenders as tfrs

import warnings
warnings.filterwarnings('ignore')

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27">🔪 Feature Selection </span>


In [ ]:
users_fg = fs.get_feature_group(
    name="users",
    version=1,
)

videos_fg = fs.get_feature_group(
    name="videos",
    version=1,
)

interactions_fg = fs.get_feature_group(
    name="interactions",
    version=1,
)

interactions_fg = fs.get_feature_group(
    name="interactions",
    version=1,
)

interactions_month_sincos_fg = fs.get_feature_group(
    name="interactions_month_sincos",
    version=1,
)

user_window_agg_1h_fg = fs.get_feature_group(
    name="user_window_agg_1h",
    version=1,
)

video_window_agg_1h_fg = fs.get_feature_group(
    name="video_window_agg_1h",
    version=1,
)

In [ ]:
QUERY_FEATURES = ["user_id", "gender", "age", "country"] 
QUERY_AGG_FEATURES =["like_count", "dislike_count", "view_count",  "total_watch_time"]

CANDIDATE_FEATURES = ["video_id", "category", "video_length"]
CANDIDATE_AGG_FEATURES = ["like_count", "dislike_count", "view_count",  "total_watch_time"]

In [ ]:
# Select features for training data
selected_features = interactions_fg.select(["id"])\
    .join(users_fg.select(QUERY_FEATURES), on="user_id")\
    .join(videos_fg.select(CANDIDATE_FEATURES), on="video_id")\
    .join(video_window_agg_1h_fg.select(CANDIDATE_AGG_FEATURES), on="video_id", prefix= "vid_")\
    .join(user_window_agg_1h_fg.select(QUERY_AGG_FEATURES), on=["user_id", "category_id"], prefix= "user_")\
    .join(interactions_month_sincos_fg.select(["month_sin", "month_cos"]), on="id")

# Uncomment this if you would like to view your selected features
#selected_features.show(5)

## <span style="color:#ff5f27">⚙️ Feature View Creation </span>


In [ ]:
feature_view = fs.get_or_create_feature_view(
    name='retrieval',
    version=1,
    query=selected_features,
)

## <span style="color:#ff5f27">🏋️ Training Dataset </span>


In [ ]:
train_df, val_df, test_df, _, _, _ = feature_view.train_validation_test_split(
    validation_size=0.1, 
    test_size=0.1,
    description='Retrieval dataset splits',
)
train_df.head(3)

You will train your retrieval model with a subset of features.

For the query embedding you will use:
- `user_id`: ID of a user.
- `gender`: Gender of a user.
- `age`: age of a user.
- `country`: country if a user.
- `month_sin`: Sine of the month of interaction date.
- `month_cos`: Cosine of the month of interaction date.
- `user_like_count`: Number of times user liked video catregory.
- `user_dislike_count`: Number of times user disliked video catregory.
- `user_view_count`: Number of times user viewed video catregory.
- `user_total_watch_time` : Total time in seconds user watched video catregory.;

For the candidate embedding you will use:
- `video_id`: ID of a video.
- `category`: Video Category.
- `vid_like_count`: Number of times video was liked by users.
- `vid_dislike_count`: Number of times video was disliked by users.
- `vid_view_count`: Number of times video was viewed by users.
- `vid_total_watch_time`: Total time in seconds video was watched. 
- `video_length`: Length of video.


In [ ]:
def df_to_ds(df):
    return tf.data.Dataset.from_tensor_slices({col: df[col] for col in df})

BATCH_SIZE = 2048
train_ds = df_to_ds(train_df).batch(BATCH_SIZE).cache().shuffle(BATCH_SIZE*10)
val_ds = df_to_ds(val_df).batch(BATCH_SIZE).cache()

In [ ]:
train_df

In [ ]:
# Query Features 
user_id_list = train_df["user_id"].unique().tolist()
countries_list = train_df["country"].unique().tolist()
gender_list = train_df["gender"].unique().tolist()

# Item Features
video_id_list = train_df["video_id"].unique().tolist()
category_list = train_df["category"].unique().tolist()

print(f"⛳️ Number of users: {len(user_id_list)}")
print(f"⛳️ Number of items: {len(video_id_list)}")

## <span style="color:#ff5f27">🏰 Two Tower Model </span>


In [ ]:
EMB_DIM = 16

In [ ]:
class QueryTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.emb_dim = EMB_DIM
        self.user_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=user_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # You add an additional embedding to account for unknown tokens.
                len(user_id_list) + 1,
                self.emb_dim
            )
        ])

        self.normalized_age = Normalization(axis=None)
        self.normalized_sin = Normalization(axis=None)
        self.normalized_cos = Normalization(axis=None)
        
        # Converts strings into integer indices (scikit-learn LabelEncoder analog)
        self.gender_tokenizer = StringLookup(
            vocabulary=gender_list,
            mask_token=None,
        )
        
        self.country_tokenizer = StringLookup(
            vocabulary=countries_list, 
            mask_token=None,
        )

        self.normalized_views = Normalization(axis=None)
        self.normalized_dislikes = Normalization(axis=None)
        self.normalized_likes = Normalization(axis=None)
        self.normalized_video_watch_time = Normalization(axis=None)
        
        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(self.emb_dim, activation="relu"),
            tf.keras.layers.Dense(self.emb_dim)
        ])

    def call(self, inputs):
        gender_embedding = tf.one_hot(
            self.gender_tokenizer(inputs["gender"]),
            len(gender_list),
        )
        
        country_embedding = tf.one_hot(
            self.country_tokenizer(inputs["country"]),
            len(countries_list),
        )
        
        concatenated_inputs = tf.concat([
            self.user_embedding(inputs["user_id"]),
            tf.reshape(self.normalized_age(inputs["age"]), (-1,1)),
            tf.reshape(self.normalized_sin(inputs["month_sin"]), (-1,1)),
            tf.reshape(self.normalized_cos(inputs["month_cos"]), (-1,1)),

            tf.reshape(self.normalized_views(inputs["user_view_count"]), (-1,1)),
            tf.reshape(self.normalized_dislikes(inputs["user_dislike_count"]), (-1,1)),
            tf.reshape(self.normalized_likes(inputs["user_like_count"]), (-1,1)),
            tf.reshape(self.normalized_video_watch_time(inputs["user_total_watch_time"]), (-1,1)),
            
            gender_embedding,
            country_embedding,
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs


query_model = QueryTower()

query_model.normalized_age.adapt(train_ds.map(lambda x : x["age"]))

# Initialize model with inputs.
query_df = train_df[QUERY_FEATURES + ["user_" + i for i in QUERY_AGG_FEATURES] + ["month_sin", "month_cos"]] 
query_ds = df_to_ds(query_df).batch(1)
query_model(next(iter(query_ds)))

In [ ]:
class ItemTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.emb_dim = EMB_DIM
        self.video_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=video_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # You add an additional embedding to account for unknown tokens.
                len(video_id_list) + 1,
                self.emb_dim,
            )
        ])
        
        # Converts strings into integer indices (scikit-learn LabelEncoder analog)
        self.category_tokenizer = StringLookup(
            vocabulary=category_list, 
            mask_token=None,
        )
        
        self.normalized_views = Normalization(axis=None)
        self.normalized_dislikes = Normalization(axis=None)
        self.normalized_likes = Normalization(axis=None)
        self.normalized_video_length = Normalization(axis=None)
        self.normalized_video_watch_time = Normalization(axis=None)

        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(self.emb_dim, activation="relu"),
            tf.keras.layers.Dense(self.emb_dim)
        ])

    def call(self, inputs):
        category_embedding = tf.one_hot(
            self.category_tokenizer(inputs["category"]),
            len(category_list),
        )

        concatenated_inputs = tf.concat([
            self.video_embedding(inputs["video_id"]),
            category_embedding,
            tf.reshape(self.normalized_views(inputs["vid_view_count"]), (-1,1)),
            tf.reshape(self.normalized_dislikes(inputs["vid_dislike_count"]), (-1,1)),
            tf.reshape(self.normalized_likes(inputs["vid_like_count"]), (-1,1)),
            tf.reshape(self.normalized_video_length(inputs["video_length"]), (-1,1)),
            tf.reshape(self.normalized_video_watch_time(inputs["vid_total_watch_time"]), (-1,1)),
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs

    
item_model = ItemTower()

item_df = train_df[CANDIDATE_FEATURES+ ["vid_" + i for i in CANDIDATE_AGG_FEATURES]]
item_df.drop_duplicates(subset="video_id", inplace=True)
item_ds = df_to_ds(item_df)

item_model(next(iter(item_ds.batch(1))))

In [ ]:
class TwoTowerModel(tf.keras.Model):
    def __init__(self, query_model, item_model):
        super().__init__()
        self.query_model = query_model
        self.item_model = item_model
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=item_ds.batch(BATCH_SIZE).map(self.item_model)
            )
        )

    def train_step(self, batch) -> tf.Tensor:
        # Set up a gradient tape to record gradients.
        with tf.GradientTape() as tape:

            # Loss computation.
            user_embeddings = self.query_model(batch)
            item_embeddings = self.item_model(batch)
            loss = self.task(
                user_embeddings, 
                item_embeddings,
                compute_metrics=False,
            )

            # Handle regularization losses as well.
            regularization_loss = sum(self.losses)

            total_loss = loss + regularization_loss

        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        metrics = {
            "loss": loss,
            "regularization_loss": regularization_loss,
            "total_loss": total_loss
        }

        return metrics

    def test_step(self, batch) -> tf.Tensor:
        # Loss computation.
        user_embeddings = self.query_model(batch)
        item_embeddings = self.item_model(batch)

        loss = self.task(
            user_embeddings, 
            item_embeddings,
            compute_metrics=False,
        )

        # Handle regularization losses as well.
        regularization_loss = sum(self.losses)

        total_loss = loss + regularization_loss

        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss

        return metrics

### <span style="color:#ff5f27">🏃🏻‍♂️ Model Training </span>

You'll train our model using the AdamW optimizer, which applies weight regularization during training.

In [ ]:
# Create a TwoTowerModel with the specified query_model and item_model
model = TwoTowerModel(query_model, item_model)

# Define an optimizer using AdamW with a learning rate of 0.01
#optimizer = tf.keras.optimizers.AdamW(
optimizer = tf.keras.optimizers.Adam(
    weight_decay=0.001, 
    learning_rate=0.01,
)

# Compile the model using the specified optimizer
model.compile(optimizer=optimizer)

In [ ]:
model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=5,
)

## <span style="color:#ff5f27">🗄️ Upload Model to Model Registry </span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

Let's connect to the model registry using the [HSML library](https://docs.hopsworks.ai/machine-learning-api/latest) from Hopsworks.

In [ ]:
mr = project.get_model_registry()

In [ ]:
class QueryModelModule(tf.Module):
    def __init__(self, query_model):
        self.query_model = query_model

    @tf.function()
    def compute_emb(self, instances):
        query_emb = self.query_model(instances)
        return {
            "user_id": instances["user_id"],
            "gender": instances["gender"],
            "age": instances["age"],
            "country": instances["country"],
            "month_sin": instances["month_sin"],
            "month_cos": instances["month_cos"],            
            "query_emb": query_emb,
        }

# wrap query_model:   query_model -> query_model_module
query_model = QueryModelModule(model.query_model)

In [ ]:
# Define the input specifications for the instances
instances_spec = {
    'user_id': tf.TensorSpec(shape=(None,), dtype=tf.string, name='user_id'),                             # Specification for user IDs
    'gender': tf.TensorSpec(shape=(None,), dtype=tf.string, name='gender'),                               # Specification for gender
    'country': tf.TensorSpec(shape=(None,), dtype=tf.string, name='country'),                             # Specification for country
    'age': tf.TensorSpec(shape=(None,), dtype=tf.int64, name='age'),                                      # Specification for age
    'user_view_count': tf.TensorSpec(shape=(None,), dtype=tf.int64, name='user_view_count'),              # Specification for view_count
    'user_dislike_count': tf.TensorSpec(shape=(None,), dtype=tf.int64, name='user_dislike_count'),        # Specification for dislike_count
    'month_sin' : tf.TensorSpec(shape=(None,), dtype=tf.float32, name='month_sin'),                       # Specification for month_sin
    'month_cos' : tf.TensorSpec(shape=(None,), dtype=tf.float32, name='month_cos'),                       # Specification for month_cos
    'user_view_count': tf.TensorSpec(shape=(None,), dtype=tf.int64, name='user_view_count'),              # Specification for view_count
    'user_dislike_count': tf.TensorSpec(shape=(None,), dtype=tf.int64, name='user_dislike_count'),        # Specification for dislike_count
    'user_like_count': tf.TensorSpec(shape=(None,), dtype=tf.int64, name='user_like_count'),              # Specification for like_count
    'user_total_watch_time': tf.TensorSpec(shape=(None,), dtype=tf.int64, name='user_total_watch_time'),  # Specification for like_count
}

# Get the concrete function for the query_model's compute_emb function using the specified input signatures
signatures = query_model.compute_emb.get_concrete_function(instances_spec)

# Save the query_model along with the concrete function signatures
tf.saved_model.save(
    query_model,           # The model to save
    "query_model",         # Path to save the model
    signatures=signatures, # Concrete function signatures to include
)

In [ ]:
tf.saved_model.save(
    model.item_model,    # The model to save
    "candidate_model",   # Path to save the model
)

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Infer input schema from data.
query_model_input_schema = Schema(query_df)

# Manually specify output schema.
query_model_output_schema = Schema([{
    "name": "query_embedding",
    "type": "float32",
    "shape": [EMB_DIM],
}])

query_model_schema = ModelSchema(
    input_schema=query_model_input_schema,
    output_schema=query_model_output_schema,
)

query_model_schema.to_dict()

In [ ]:
# Sample a query example from the query DataFrame
query_example = query_df.sample().to_dict("records")

# Create a tensorflow model for the query_model in the Model Registry 
mr_query_model = mr.tensorflow.create_model(
    name="query_model",                                           # Name of the model
    description="Model that generates query embeddings from user features",  # Description of the model
    input_example=query_example,                                  # Example input for the model
    model_schema=query_model_schema,                              # Schema of the model
)

# Save the query_model to the Model Registry
mr_query_model.save("query_model")                                # Path to save the model

In [ ]:
# Define the input schema for the candidate_model based on item_df
candidate_model_input_schema = Schema(item_df)

# Define the output schema for the candidate_model, specifying the shape and type of the output
candidate_model_output_schema = Schema([{
    "name": "candidate_embedding",   # Name of the output feature
    "type": "float32",               # Data type of the output feature
    "shape": [EMB_DIM],              # Shape of the output feature
}])

# Combine the input and output schemas to create the overall model schema for the candidate_model
candidate_model_schema = ModelSchema(
    input_schema=candidate_model_input_schema,    # Input schema for the model
    output_schema=candidate_model_output_schema,  # Output schema for the model
)

# Sample a candidate example from the item DataFrame
candidate_example = item_df.sample().to_dict("records")

# Create a tensorflow model for the candidate_model in the Model Registry
mr_candidate_model = mr.tensorflow.create_model(
    name="candidate_model",                                        # Name of the model
    description="Model that generates candidate embeddings from video features",  # Description of the model
    input_example=candidate_example,                              # Example input for the model
    model_schema=candidate_model_schema,                          # Schema of the model
)

# Save the candidate_model to the Model Registry
mr_candidate_model.save("candidate_model")                        # Path to save the model

---